In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import v2

FLIP_PROBABILITY = 0.1

data_transform = v2.Compose([
    v2.Resize(size=(224,224)),
    v2.RandomHorizontalFlip(p=0.5),
    v2.RandomRotation(degrees=(-10,10)),
    v2.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.05, hue=0.05),
    v2.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    v2.ToTensor(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Add normalization
])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
train_data = datasets.ImageFolder(root="/content/drive/MyDrive/dataset_v3/train",
                                  transform=data_transform,
                                  target_transform=None)
test_data = datasets.ImageFolder(root="/content/drive/MyDrive/dataset_v3/test",
                                  transform=data_transform)
class_names = train_data.classes

In [5]:
from sklearn.utils import resample

def balance_dataset(dataset, size):
    # Separate data by class
    class_data = {i: [] for i in range(8)}  # For 8 classes

    # Group data by class
    for idx in range(len(dataset)):
        _, label = dataset[idx]
        class_data[label].append(idx)

    # Randomly sample indices from each class
    balanced_indices = []
    for class_idx, indices in class_data.items():
        # If class has more samples than min_size, downsample it
        if len(indices) > size:
            balanced_indices.extend(np.random.choice(indices, size=size, replace=False))
        else:
            balanced_indices.extend(indices)

    # Create a subset dataset
    from torch.utils.data import Subset
    balanced_dataset = Subset(dataset, balanced_indices)

    return balanced_dataset

# Use it like this:
balanced_train_data = balance_dataset(train_data, 200)

balanced_test_data = balance_dataset(test_data, 50)

In [6]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16
train_dataloader = DataLoader(balanced_train_data, batch_size=BATCH_SIZE, shuffle=True)
test_dataloader = DataLoader(balanced_test_data, batch_size=BATCH_SIZE, shuffle=False)

In [7]:
train_transform = v2.Compose([
    v2.Resize(size=(224,224)),
    v2.TrivialAugmentWide(num_magnitude_bins=5),
    v2.ToTensor()
])

test_transform = v2.Compose([
    v2.Resize(size=(224,224)),
    v2.ToTensor()
])

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [8]:
import torch
import torch.nn as nn

class PatchEmbed(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, embed_dim=768):
        super().__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.n_patches = (img_size // patch_size) ** 2

        self.proj = nn.Conv2d(
            in_channels,
            embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )

    def forward(self, x):
        x = self.proj(x)  # (B, E, H', W')
        x = x.flatten(2)  # (B, E, N)
        x = x.transpose(1, 2)  # (B, N, E)
        return x

In [9]:
class Attention(nn.Module):
    def __init__(self, dim, n_heads=12, qkv_bias=True, attn_drop=0.1, proj_drop=0.1):
        super().__init__()
        self.n_heads = n_heads
        self.scale = (dim // n_heads) ** -0.5

        self.qkv = nn.Linear(dim, dim * 3, bias=qkv_bias)
        self.attn_drop = nn.Dropout(attn_drop)
        self.proj = nn.Linear(dim, dim)
        self.proj_drop = nn.Dropout(proj_drop)

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.n_heads, C // self.n_heads).permute(2, 0, 3, 1, 4)
        q, k, v = qkv[0], qkv[1], qkv[2]

        attn = (q @ k.transpose(-2, -1)) * self.scale
        attn = attn.softmax(dim=-1)
        attn = self.attn_drop(attn)

        x = (attn @ v).transpose(1, 2).reshape(B, N, C)
        x = self.proj(x)
        x = self.proj_drop(x)
        return x

In [10]:
class MLP(nn.Module):
    def __init__(self, in_features, hidden_features, out_features, drop=0.1):
        super().__init__()
        self.fc1 = nn.Linear(in_features, hidden_features)
        self.act = nn.GELU()
        self.fc2 = nn.Linear(hidden_features, out_features)
        self.drop = nn.Dropout(drop)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.drop(x)
        x = self.fc2(x)
        x = self.drop(x)
        return x

In [11]:
class Block(nn.Module):
    def __init__(self, dim, n_heads, mlp_ratio=4., qkv_bias=True, drop=0.1, attn_drop=0.1):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = Attention(dim, n_heads=n_heads, qkv_bias=qkv_bias, attn_drop=attn_drop, proj_drop=drop)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = MLP(dim, int(dim * mlp_ratio), dim, drop)

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x

In [12]:
class VisionTransformer(nn.Module):
    def __init__(self, img_size=224, patch_size=16, in_channels=3, num_classes=len(class_names),
                 embed_dim=768, depth=12, n_heads=12, mlp_ratio=4., qkv_bias=True,
                 drop_rate=0.1, attn_drop_rate=0.1):
        super().__init__()
        self.patch_embed = PatchEmbed(img_size, patch_size, in_channels, embed_dim)
        num_patches = self.patch_embed.n_patches

        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches + 1, embed_dim))
        torch.nn.init.trunc_normal_(self.pos_embed, std=0.02)
        torch.nn.init.trunc_normal_(self.cls_token, std=0.02)
        self.pos_drop = nn.Dropout(drop_rate)

        self.blocks = nn.Sequential(*[
            Block(embed_dim, n_heads, mlp_ratio, qkv_bias, drop_rate, attn_drop_rate)
            for _ in range(depth)
        ])

        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)

        cls_token = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_token, x), dim=1)
        x = x + self.pos_embed
        x = self.pos_drop(x)

        x = self.blocks(x)
        x = self.norm(x)

        x = x[:, 0]  # take cls token
        x = self.head(x)
        return x

In [13]:
import torch
from torch.nn import functional as F
from tqdm import tqdm
import numpy as np
from sklearn.metrics import accuracy_score

class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def train_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    losses = AverageMeter()
    progress_bar = tqdm(train_loader, desc='Training')

    all_predictions = []
    all_labels = []

    for batch in progress_bar:
        images, labels = batch
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        losses.update(loss.item(), images.size(0))
        progress_bar.set_postfix({'train_loss': f'{losses.avg:.4f}'})

        predictions = torch.argmax(outputs, dim=1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    epoch_accuracy = accuracy_score(all_labels, all_predictions)
    return losses.avg, epoch_accuracy

def evaluate(model, test_loader, criterion, device):
    model.eval()
    losses = AverageMeter()

    all_predictions = []
    all_labels = []

    with torch.no_grad():
        progress_bar = tqdm(test_loader, desc='Testing')
        for batch in progress_bar:
            images, labels = batch
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            losses.update(loss.item(), images.size(0))
            progress_bar.set_postfix({'test_loss': f'{losses.avg:.4f}'})

            predictions = torch.argmax(outputs, dim=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = accuracy_score(all_labels, all_predictions)
    return losses.avg, accuracy

def train_model(model, train_loader, test_loader, num_epochs=100):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=5e-5,
        weight_decay=0.02,
        betas=(0.9, 0.999)
    )
    steps_per_epoch = len(train_loader)
    total_steps = steps_per_epoch * num_epochs
    scheduler = torch.optim.lr_scheduler.OneCycleLR(
        optimizer,
        max_lr=5e-4,
        total_steps=total_steps,
        pct_start=0.2,
        anneal_strategy='cos',
        cycle_momentum=True,
        base_momentum=0.85,
        max_momentum=0.95,
        div_factor=10.0,
        final_div_factor=1000.0
    )

    # Add tracking for best model
    best_accuracy = 0.0
    best_model_state = None
    best_epoch = 0
    best_test_loss = float('inf')

    print("Starting training...")
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        train_loss, train_accuracy = train_epoch(
            model, train_loader, criterion, optimizer, device
        )
        test_loss, test_accuracy = evaluate(
            model, test_loader, criterion, device
        )
        scheduler.step()

        # Save best model
        if test_accuracy > best_accuracy:
            best_accuracy = test_accuracy
            best_test_loss = test_loss
            best_model_state = model.state_dict().copy()
            best_epoch = epoch + 1
            print(f"New best model saved! Accuracy: {best_accuracy:.4f}")

        if epoch % 5 == 0:
            print(f"Train Loss: {train_loss:.4f} | Train Accuracy: {train_accuracy:.4f}")
            print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f}")

    # Load best model before returning
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print(f"\nLoaded best model from epoch {best_epoch}")
        print(f"Best Test Metrics:")
        print(f"Test Loss: {best_test_loss:.4f}")
        print(f"Test Accuracy: {best_accuracy:.4f}")

    return model, best_accuracy

# Initialize model
model = VisionTransformer(
    img_size=224,
    patch_size=16,
    in_channels=3,
    num_classes=len(class_names),  # set this to your number of classes
    embed_dim=384,
    depth=8,
    n_heads=8,
    mlp_ratio=4,
    qkv_bias=True,
    drop_rate=0.15
)

# Train the model
trained_model, best_accuracy = train_model(
    model,
    train_dataloader,
    test_dataloader,
    num_epochs=100
)
print(f"\nTraining completed!")
print(f"Best test accuracy achieved: {best_accuracy:.4f}")

Starting training...

Epoch 1/100


Testing: 100%|██████████| 17/17 [00:03<00:00,  4.45it/s, test_loss=2.0483]


New best model saved! Accuracy: 0.2355
Train Loss: 2.0345 | Train Accuracy: 0.2434
Test Loss: 2.0483 | Test Accuracy: 0.2355

Epoch 2/100


Testing: 100%|██████████| 17/17 [00:03<00:00,  4.40it/s, test_loss=1.9282]


New best model saved! Accuracy: 0.3050

Epoch 3/100


...


Epoch 99/100


Testing: 100%|██████████| 17/17 [00:04<00:00,  3.79it/s, test_loss=1.1110]



Epoch 100/100


Testing: 100%|██████████| 17/17 [00:04<00:00,  4.15it/s, test_loss=1.1176]


Loaded best model from epoch 94
Best Test Metrics:
Test Loss: 0.9997
Test Accuracy: 0.8224

Training completed!
Best test accuracy achieved: 0.8224
